<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name:</b> DeepLearning19 - Deodato Giacomo - Bucquet Anthime
 
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from time import time

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

print("Image Shape: {}".format(X_train[0].shape))
print("Training Set: {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set: {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set: 55000 samples
Validation Set: 5000 samples
Test Set: 10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
tf.reset_default_graph() # reset the default graph before defining a new model

#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288568147
Epoch:  02   =====> Loss= 0.732308971
Epoch:  03   =====> Loss= 0.600200272
Epoch:  04   =====> Loss= 0.536473810
Epoch:  05   =====> Loss= 0.497731537
Epoch:  06   =====> Loss= 0.470901849
Epoch:  07   =====> Loss= 0.451301165
Epoch:  08   =====> Loss= 0.435680064
Epoch:  09   =====> Loss= 0.423353645
Epoch:  10   =====> Loss= 0.413345681
Epoch:  11   =====> Loss= 0.404281634
Epoch:  12   =====> Loss= 0.396839049
Epoch:  13   =====> Loss= 0.390208097
Epoch:  14   =====> Loss= 0.384563916
Epoch:  15   =====> Loss= 0.379514001
Epoch:  16   =====> Loss= 0.374462080
Epoch:  17   =====> Loss= 0.370331678
Epoch:  18   =====> Loss= 0.366488553
Epoch:  19   =====> Loss= 0.362964171
Epoch:  20   =====> Loss= 0.359689518
Epoch:  21   =====> Loss= 0.356679378
Epoch:  22   =====> Loss= 0.353974787
Epoch:  23   =====> Loss= 0.351341481
Epoch:  24   =====> Loss= 0.348637887
Epoch:  25   =====> Loss= 0.346562084
Epoch:  26   =====> Loss= 0.344416684
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 
<img src="MNIST_figures/plot1.png" align="center" width="80%">

# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="http://eblearn.sourceforge.net/lib/exe/lenet5.png" width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [2]:
def LeNet5(X):    
    
    def weight_variable(shape):
        return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    def bias_variable(shape):
        return tf.Variable(tf.constant(0.05, shape=[shape]))
    
    X = tf.reshape(X, [-1, 28, 28, 1])
    conv1 = tf.nn.conv2d(input=X,
                         filter=weight_variable([5, 5, 1, 6]),
                         strides=[1, 1, 1, 1],
                         padding='SAME')
    conv1 = tf.nn.relu(tf.nn.bias_add(conv1, bias_variable(6)))
    pool1 = tf.nn.max_pool(value=conv1,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID')
    conv2 = tf.nn.conv2d(input=pool1,
                         filter=weight_variable([5, 5, 6, 16]),
                         strides=[1, 1, 1, 1],
                         padding='VALID')
    conv2 = tf.nn.relu(tf.nn.bias_add(conv2, bias_variable(16)))
    pool2 = tf.nn.max_pool(value=conv2,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID')
    flatten = tf.contrib.layers.flatten(pool2)
    fc1 = tf.nn.bias_add(tf.matmul(flatten, weight_variable([400, 120])), bias_variable(120))
    fc1 = tf.nn.relu(fc1)
    fc2 = tf.nn.bias_add(tf.matmul(fc1, weight_variable([120,84])), bias_variable(84))
    fc2 = tf.nn.relu(fc2)
    fc3 = tf.nn.bias_add(tf.matmul(fc2, weight_variable([84,10])), bias_variable(10))
    fc3 = tf.nn.softmax(fc3)
    return fc3

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

conv1: $5\ x\ 5\ x\ 1\ x\ 6 + 6 = 156$

conv2: $5\ x\ 5\ x\ 6\ x\ 16 + 16 = 2416$

fc1: $400\ x\ 120 + 120 = 48120$

fc2: $120\ x\ 84 + 84 = 10164$

fc3: $84\ x\ 10 + 10 = 850$

total: $61706$

In [6]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    #print(shape)
    #print(len(shape))
    variable_parameters = 1
    for dim in shape:
        #print(dim)
        variable_parameters *= dim.value
    #print(variable_parameters)
    total_parameters += variable_parameters
print("Number of parameters of this model {} ".format(total_parameters))

Number of parameters of this model 61706 


<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [3]:
# reset the default graph before defining a new model
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')

learning_rate = 0.001
n_epochs = 40
batch_size = 128
verbose = 2
logs_path = 'log_files/'
saving_path = 'models/'

with tf.name_scope('Network'):
    model = LeNet5(x)
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(model, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [4]:
def evaluate(Y_pred, Y_true):
    acc = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y_true, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    return acc

In [5]:
with tf.name_scope('Accuracy'):
    accuracy = evaluate(model, y)

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [7]:
# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost and accuracy tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
tf.summary.scalar("Accuracy_LeNet-5_SGD", accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [8]:
def train_mnist(n_epochs, batch_size, cost, optimizer, accuracy, verbose=100):
    start = time()
    for epoch in range(n_epochs):
        avg_cost = 0.
        n_batches = int(mnist.train.num_examples/batch_size)

        for i in range(n_batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))

            _, c, summary = sess.run([optimizer, cost, merged_summary_op], feed_dict={x: batch_xs, y: batch_ys})

            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * n_batches + i)

            # Compute average loss
            avg_cost += c

        if verbose != 0 and ((epoch + 1) % verbose == 0):
            print("Epoch: {:3d} | Loss: {:.5f} | Accuracy: {:.2f} %"\
                  .format(epoch+1, avg_cost/n_batches, accuracy.eval(feed_dict={x: mnist.validation.images, y: mnist.validation.labels})*100))
        end = time() - start
    print("\n Training Time: {} min {} s".format(int(end/60), int(end%60)))

    

In [9]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    train_mnist(n_epochs, batch_size, cost, optimizer, accuracy, verbose)
    print("Accuracy on the test data: {:.2f} %"\
          .format(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})*100))
    summary_writer.flush()

Epoch:   2 | Loss: 2.12917 | Accuracy: 49.56 %
Epoch:   4 | Loss: 1.37251 | Accuracy: 71.74 %
Epoch:   6 | Loss: 0.66525 | Accuracy: 83.96 %
Epoch:   8 | Loss: 0.46188 | Accuracy: 87.54 %
Epoch:  10 | Loss: 0.38557 | Accuracy: 89.32 %
Epoch:  12 | Loss: 0.34211 | Accuracy: 90.66 %
Epoch:  14 | Loss: 0.31145 | Accuracy: 91.52 %
Epoch:  16 | Loss: 0.28748 | Accuracy: 92.36 %
Epoch:  18 | Loss: 0.26742 | Accuracy: 92.86 %
Epoch:  20 | Loss: 0.25058 | Accuracy: 93.26 %
Epoch:  22 | Loss: 0.23602 | Accuracy: 93.82 %
Epoch:  24 | Loss: 0.22309 | Accuracy: 94.08 %
Epoch:  26 | Loss: 0.21177 | Accuracy: 94.38 %
Epoch:  28 | Loss: 0.20186 | Accuracy: 94.72 %
Epoch:  30 | Loss: 0.19270 | Accuracy: 94.78 %
Epoch:  32 | Loss: 0.18458 | Accuracy: 95.20 %
Epoch:  34 | Loss: 0.17719 | Accuracy: 95.24 %
Epoch:  36 | Loss: 0.17071 | Accuracy: 95.46 %
Epoch:  38 | Loss: 0.16435 | Accuracy: 95.58 %
Epoch:  40 | Loss: 0.15865 | Accuracy: 95.74 %

 Training Time: 11 min 19 s
Accuracy on the test data: 95.6

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

#### Accuracy
<img src="MNIST_figures/plot2.png" align="center" width="80%">
#### Loss
<img src="MNIST_figures/plot3.png" align="center" width="80%">

<b> Part 2 </b> : LeNET 5 Optimization

<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table below:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |        95.6%       |        99.11 %         |       
| Training Time        |         11 min 19 s       |       12 min 5 s          |  

In [10]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')

model = LeNet5(x)

learning_rate = 0.001
n_epochs = 40
batch_size = 128
verbose = 2
logs_path = 'log_files/'
saving_path = 'models/'

with tf.name_scope('Loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(model, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('AdamOpti'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    accuracy = evaluate(model, y)

In [11]:
# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost and accuracy tensors
tf.summary.scalar("Loss_LeNet-5_ADAM", cost)
tf.summary.scalar("Accuracy_LeNet-5_ADAM", accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    train_mnist(n_epochs, batch_size, cost, optimizer, accuracy, verbose)
    print("Accuracy on the test data: {:.2f} %"\
          .format(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})*100))
    summary_writer.flush()


Epoch:   2 | Loss: 0.09158 | Accuracy: 97.72 %
Epoch:   4 | Loss: 0.04850 | Accuracy: 98.56 %
Epoch:   6 | Loss: 0.03456 | Accuracy: 98.38 %
Epoch:   8 | Loss: 0.02495 | Accuracy: 98.70 %
Epoch:  10 | Loss: 0.01857 | Accuracy: 98.94 %
Epoch:  12 | Loss: 0.01549 | Accuracy: 98.66 %
Epoch:  14 | Loss: 0.01234 | Accuracy: 98.98 %
Epoch:  16 | Loss: 0.01013 | Accuracy: 98.72 %
Epoch:  18 | Loss: 0.00849 | Accuracy: 99.22 %
Epoch:  20 | Loss: 0.00822 | Accuracy: 99.10 %
Epoch:  22 | Loss: 0.00671 | Accuracy: 98.92 %
Epoch:  24 | Loss: 0.00610 | Accuracy: 98.84 %
Epoch:  26 | Loss: 0.00760 | Accuracy: 99.04 %
Epoch:  28 | Loss: 0.00422 | Accuracy: 98.78 %
Epoch:  30 | Loss: 0.00459 | Accuracy: 98.68 %
Epoch:  32 | Loss: 0.00609 | Accuracy: 98.72 %
Epoch:  34 | Loss: 0.00443 | Accuracy: 99.14 %
Epoch:  36 | Loss: 0.00522 | Accuracy: 99.20 %
Epoch:  38 | Loss: 0.00367 | Accuracy: 99.02 %
Epoch:  40 | Loss: 0.00654 | Accuracy: 99.16 %

 Training Time: 12 min 5 s
Accuracy on the test data: 99.11

#### Accuracy
<img src="MNIST_figures/plot4.png" align="center" width="80%">
#### Loss
<img src="MNIST_figures/plot5.png" align="center" width="80%">

<div class="alert alert-info">
Which optimizer gives the best accuracy on test data?
</div>

<div class="alert alert-success">
Adam combines the benefits of two other optimizers: AdaGrad and RMSProp, instead of adapting the learning rate based on the average first moment (the mean) as in RMSProp, Adam also makes use of the average of the second moments of the gradients (the uncentered variance).<br><br>
The main down side of the algorithm is that it requires more computation to be performed for each parameter in each training step because it maintains the moving averages and variance, and calculates the scaled gradient, in fact it takes almost one minute more time than normal Gradient Descent to train for 40 epochs.
</div>

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

In [14]:
def LeNet5_dropout(X):  
    
    
    def weight_variable(shape):
        return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    def bias_variable(shape):
        return tf.Variable(tf.constant(0.05, shape=[shape]))
    
    X = tf.reshape(X, [-1, 28, 28, 1])
    conv1 = tf.nn.conv2d(input=X,
                         filter=weight_variable([5, 5, 1, 6]),
                         strides=[1, 1, 1, 1],
                         padding='SAME')
    conv1 = tf.nn.relu(tf.nn.bias_add(conv1, bias_variable(6)))
    pool1 = tf.nn.max_pool(value=conv1,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID')
    conv2 = tf.nn.conv2d(input=pool1,
                         filter=weight_variable([5, 5, 6, 16]),
                         strides=[1, 1, 1, 1],
                         padding='VALID')
    conv2 = tf.nn.relu(tf.nn.bias_add(conv2, bias_variable(16)))
    pool2 = tf.nn.max_pool(value=conv2,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID')
    flatten = tf.contrib.layers.flatten(pool2)
    dropout = tf.nn.dropout(flatten, 0.75)
    fc1 = tf.nn.bias_add(tf.matmul(dropout, weight_variable([400, 120])), bias_variable(120))
    fc1 = tf.nn.relu(fc1)
    fc2 = tf.nn.bias_add(tf.matmul(fc1, weight_variable([120,84])), bias_variable(84))
    fc2 = tf.nn.relu(fc2)
    fc3 = tf.nn.bias_add(tf.matmul(fc2, weight_variable([84,10])), bias_variable(10))
    fc3 = tf.nn.softmax(fc3)
    return fc3

In [15]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='predict_label')

model_dropout = LeNet5_dropout(x)

learning_rate = 0.001
n_epochs = 100
batch_size = 128
verbose = 5
logs_path = 'log_files/'
saving_path = 'models/'

with tf.name_scope('Loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(model_dropout, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('AdamOpti'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    accuracy = evaluate(model_dropout, y)

In [16]:
# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost and accuracy tensors
tf.summary.scalar("Loss_LeNet-5_ADAM_Drop", cost)
tf.summary.scalar("Accuracy_LeNet-5_ADAM_drop", accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    train_mnist(n_epochs, batch_size, cost, optimizer, accuracy, verbose)
    print("Accuracy on the test data: {:.2f} %"\
          .format(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})*100))
    summary_writer.flush()

Epoch:   5 | Loss: 0.05678 | Accuracy: 98.04 %
Epoch:  10 | Loss: 0.03534 | Accuracy: 98.48 %
Epoch:  15 | Loss: 0.02524 | Accuracy: 98.76 %
Epoch:  20 | Loss: 0.02025 | Accuracy: 98.88 %
Epoch:  25 | Loss: 0.01513 | Accuracy: 98.82 %
Epoch:  30 | Loss: 0.01316 | Accuracy: 98.80 %
Epoch:  35 | Loss: 0.01228 | Accuracy: 98.80 %
Epoch:  40 | Loss: 0.01114 | Accuracy: 98.94 %
Epoch:  45 | Loss: 0.01060 | Accuracy: 99.12 %
Epoch:  50 | Loss: 0.01009 | Accuracy: 98.86 %
Epoch:  55 | Loss: 0.00897 | Accuracy: 98.90 %
Epoch:  60 | Loss: 0.00757 | Accuracy: 98.96 %
Epoch:  65 | Loss: 0.00743 | Accuracy: 98.96 %
Epoch:  70 | Loss: 0.00671 | Accuracy: 99.06 %
Epoch:  75 | Loss: 0.00605 | Accuracy: 98.82 %
Epoch:  80 | Loss: 0.00536 | Accuracy: 99.04 %
Epoch:  85 | Loss: 0.00554 | Accuracy: 99.00 %
Epoch:  90 | Loss: 0.00593 | Accuracy: 98.96 %
Epoch:  95 | Loss: 0.00480 | Accuracy: 98.92 %
Epoch: 100 | Loss: 0.00541 | Accuracy: 98.70 %

 Training Time: 37 min 46 s
Accuracy on the test data: 98.8

#### Accuracy
<img src="MNIST_figures/plot6.png" align="center" width="80%">
#### Loss
<img src="MNIST_figures/plot7.png" align="center" width="80%">

<div class="alert alert-success">
Of course, the accuracy is not yet as high as for the other implementations of the model because when using dropout some neurons are stochastically dropped out from the training, therefore it takes more time to obtain the same results but the network is more robust and so, less prone to overfitting.
</div>